In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

In [3]:
import csv
import os
import dask
import dask.dataframe as dd
import fastparquet
import pandas as pd
import altair as alt
from datetime import datetime, timedelta

In [81]:
df = dd.read_csv(os.path.join('data', 'yellow_tripdata_2010-*.csv'),
                 parse_dates=['pickup_datetime','dropoff_datetime'],
                 quoting=csv.QUOTE_NONE, encoding='utf-8', error_bad_lines=False,
                 dtype={'trip_distance':'float64', 'store_and_fwd_flag':'object'})


In [82]:
df.tail()



,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
220375,VTS,2010-12-05 01:56:00,2010-12-05 02:02:00,2,1.3,-73.983850,40.749142,1,NaN,-73.969747,40.756967,CRD,5.7,0.5,0.5,1.00,0.0,7.70
220376,CMT,2010-12-02 01:11:19,2010-12-02 01:14:53,1,1.4,-73.957606,40.717869,1,N,-73.955151,40.735764,CRD,5.3,0.5,0.5,1.26,0.0,7.56
220377,CMT,2010-12-02 01:07:41,2010-12-02 01:39:34,2,12.1,-73.989767,40.729652,1,N,-74.033006,40.618895,CRD,30.9,0.5,0.5,6.38,0.0,38.28
220378,CMT,2010-12-02 00:44:45,2010-12-02 00:54:41,1,2.6,-73.991002,40.740599,1,N,-73.998046,40.746860,CRD,8.9,0.5,0.5,1.48,0.0,11.38
220379,CMT,2010-12-02 00:17:46,2010-12-02 00:23:37,1,1.6,-74.005401,40.744262,1,N,-73.987623,40.760235,CRD,6.5,0.5,0.5,1.00,0.0,8.50


In [ ]:
#df.trip_distance.max().compute()
#df['store_and_fwd_flag'].head()


In [ ]:
#df[df['store_and_fwd_flag'] == '*'].head()
#dask.compute(df.trip_distance.max().compute())

In [ ]:
#len(df)


In [ ]:
def plot1wWindow(df, rainBoundaryTS):
    b = rainBoundaryTS

    df_d = df[(df['pickup_datetime'] >= b + timedelta(days=-.25)) & (df['pickup_datetime'] < b + timedelta(days=.25))]

    df_d.to_parquet('./data/window_02.parquet', write_index=False)
    df_win01 = pd.read_parquet('./data/window_02.parquet', engine='fastparquet')

    # https://stackoverflow.com/questions/3463930/how-to-round-the-minute-of-a-datetime-object
    df_win01['pickup_datetime_1min'] =   df_win01['pickup_datetime'].dt.round('1min')

    df_rpm = df_win01.groupby('pickup_datetime_1min', as_index=False).agg({'vendor_id':'count'}).iloc[1:-1,:]

    alt.data_transformers.disable_max_rows()


    line = alt.Chart(pd.DataFrame({'x': [b]})).mark_rule().encode(x=alt.X('x:T', title='') )

    rides = alt.Chart(df_rpm, title="Rides around " + str(b)).mark_line().encode(
        x=alt.X('pickup_datetime_1min:T', title='Date/time'),        
        y=alt.Y('vendor_id:Q', title='Rides')
        #color='vendor_id:Q'
    ).properties(
    width=700,
        height=400
    ).interactive()

    return line+rides
    
    


In [ ]:
plot1wWindow(df, datetime(2010, 1, 17, 11, 51, 0))

In [ ]:
# Good
# df_d = df[(df['pickup_datetime'] >= '2010-01-17 10:51:00') & (df['pickup_datetime'] < '2010-01-17 13:51:00')]

# b = datetime(2010, 4, 27, 9, 0, 0)

# big spike
# b = datetime(2010, 6, 17, 21, 0, 0)

b = datetime(2010, 6, 16, 21, 0, 0)
b = datetime(2010, 4, 27, 9, 0, 0)


In [ ]:
#df_win01 = pd.read_parquet('./data/window_01.parquet', engine='fastparquet')
df_win01 = pd.read_parquet('./data/window_02.parquet', engine='fastparquet')


In [ ]:
df.tail()

In [ ]:
b = datetime(2010, 1, 17, 11, 51, 0)

In [ ]:

# df_d = df[(df['pickup_datetime'] >= b + timedelta(days=-.25)) & (df['pickup_datetime'] < b + timedelta(days=.25))]

df_d = df[ \
          ((df['pickup_datetime'] >= (b + timedelta(days=-.25-7))) & (df['pickup_datetime'] < (b + timedelta(days=.25-7)))) \
          | ((df['pickup_datetime'] >= (b + timedelta(days=-.25))) & (df['pickup_datetime'] < (b + timedelta(days=.25)))) \
          | ((df['pickup_datetime'] >= (b + timedelta(days=-.25+7))) & (df['pickup_datetime'] < (b + timedelta(days=.25+7))))         
         ]

df_d = df_d.compute()


In [ ]:
df_d['pickup_datetime_1min'] = df_d['pickup_datetime'].dt.round('1min')

df_3w = df_d.groupby('pickup_datetime_1min', as_index=False).agg({'vendor_id':'count'}).iloc[:,:]

df_3w.shape


In [ ]:
df_3w[(df_3w['pickup_datetime_1min'] >= (b + timedelta(days=-.25))) & (df_3w['pickup_datetime_1min'] <= (b + timedelta(days=.25)))]

In [ ]:
df_3w[df_3w['pickup_datetime_1min'] < (b + timedelta(days=-.25))]

In [ ]:
df_3w[df_3w['pickup_datetime_1min'] > (b + timedelta(days=+.25))]

In [ ]:
def parse_weeks(x):
    if x < (b + timedelta(days=-.25)):
       return -1
    elif x > (b + timedelta(days=+.25)):
       return 1
    else:
       return 0

df_3w['which_week'] = df_3w['pickup_datetime_1min'].apply(parse_weeks)


df_3w.groupby('which_week')['vendor_id'].count()

In [ ]:
df_3w

In [ ]:
# https://stackoverflow.com/questions/31226142/python-pandas-delete-the-first-row-by-group
df_3wfl = df_3w.groupby('which_week').apply(lambda group: group.iloc[1:-1, :])
#df_3w.columns = ['pickup_datetime_1min', 'vendor_id', 'which_week2']

# https://thispointer.com/pandas-delete-last-column-of-dataframe-in-python/
df_3wfl = df_3wfl.iloc[: , :-1].reset_index()
del df_3wfl['level_1']
df_3wfl

In [ ]:
# https://stackoverflow.com/questions/29353096/add-a-sequence-number-to-each-element-in-a-group-using-python
df_3wfl['sequence']=df_3wfl.groupby('which_week').cumcount()+1
df_3wfl

In [ ]:
alt.data_transformers.disable_max_rows()

line = alt.Chart(pd.DataFrame({'x': [b + timedelta(days=-7), b, b + timedelta(days=7)]})).mark_rule().encode(x=alt.X('x:T', title='') )

rides = alt.Chart(df_3w, title="Rides around " + str(b)).mark_line().encode(
    x=alt.X('pickup_datetime_1min:T', title='Date/time'),        
    y=alt.Y('vendor_id:Q', title='Rides')
    #color='vendor_id:Q'
).properties(
width=700,
    height=400
).interactive()

line+rides

In [ ]:
alt.data_transformers.disable_max_rows()

line = alt.Chart(pd.DataFrame({'x': [360]})).mark_rule().encode(x=alt.X('x:O', title='') )

rides = alt.Chart(df_3wfl, title="Rides around " + str(b)).mark_line().encode(
    x=alt.X('sequence:O', title='Date/time'),        
    y=alt.Y('vendor_id:Q', title='Rides'),
    color='which_week:N'
).properties(
width=700,
    height=400
).interactive()

rides+line

In [65]:
df['pickup_datetime_stdtime'] = df['pickup_datetime'].apply(lambda x: x + timedelta(hours=-1) if datetime(2010, 3, 14, 3, 0, 0) <= x < datetime(2010, 11, 7, 1, 0, 0) else x, meta=('pickup_datetime', 'datetime64[ns]'))

In [66]:
df[(df['pickup_datetime']>datetime(2010, 3, 14, 3, 10, 0) ) & (df['pickup_datetime']<datetime(2010, 3, 14, 3, 10, 7) )].compute()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_datetime_stdtime
201687,CMT,2010-03-14 03:10:03,2010-03-14 03:12:44,2,0.8,-73.989602,40.720547,1,0,-73.977869,40.721082,Cas,4.1,0.5,0.5,0.00,0.0,5.10,2010-03-14 02:10:03
261744,CMT,2010-03-14 03:10:01,2010-03-14 03:18:58,2,2.9,-74.001733,40.745826,1,0,-74.015303,40.709791,Cre,9.3,0.5,0.5,1.00,0.0,11.30,2010-03-14 02:10:01
120709,CMT,2010-03-14 03:10:06,2010-03-14 03:18:12,1,2.0,-73.990898,40.736742,1,0,-74.009167,40.713969,Cas,7.3,0.5,0.5,0.00,0.0,8.30,2010-03-14 02:10:06
210080,CMT,2010-03-14 03:10:04,2010-03-14 03:14:53,2,1.2,-74.001511,40.729271,1,0,-73.999827,40.716662,Cas,5.3,0.5,0.5,0.00,0.0,6.30,2010-03-14 02:10:04
102439,CMT,2010-03-14 03:10:05,2010-03-14 03:21:19,1,3.7,-73.991644,40.759931,1,0,-73.948175,40.782976,Cas,10.5,0.5,0.5,0.00,0.0,11.50,2010-03-14 02:10:05
104784,CMT,2010-03-14 03:10:01,2010-03-14 03:15:30,1,1.4,-73.988459,40.723382,1,0,-73.986583,40.738807,Cre,5.7,0.5,0.5,1.00,0.0,7.70,2010-03-14 02:10:01
109509,CMT,2010-03-14 03:10:02,2010-03-14 03:13:51,1,0.4,-74.004111,40.719716,1,0,-73.999377,40.724991,Cas,4.1,0.5,0.5,0.00,0.0,5.10,2010-03-14 02:10:02
198999,CMT,2010-03-14 03:10:03,2010-03-14 03:25:52,1,4.1,-73.987226,40.755736,1,0,-74.012771,40.702106,Cas,12.1,0.5,0.5,0.00,0.0,13.10,2010-03-14 02:10:03
286790,CMT,2010-03-14 03:10:04,2010-03-14 03:34:12,1,17.1,-73.781184,40.645076,1,0,-73.972054,40.745601,Cre,45.0,0.0,0.5,6.00,5.5,57.00,2010-03-14 02:10:04
21318,CMT,2010-03-14 03:10:06,2010-03-14 03:20:22,2,3.9,-73.966860,40.760893,1,0,-73.944405,40.806275,Cre,10.5,0.5,0.5,1.72,0.0,13.22,2010-03-14 02:10:06


In [83]:
from datetime import datetime

start_time  = datetime.now()
print("Starting: " + str(start_time))



orig_boundary_list = ['2010-01-17T11:51:00',
                     '2010-01-24T15:05:00',
                     '2010-01-25T06:01:00',
                     '2010-01-25T16:51:00',
                      '2010-01-26T00:51:00',
                      '2010-02-23T06:51:00',
                      '2010-02-25T02:51:00',
                      '2010-03-12T07:51:00',
                      '2010-03-12T11:51:00',
                      '2010-03-14T09:26:00',
                      '2010-03-15T00:28:00',
                      '2010-03-22T05:00:00',
                      '2010-03-23T14:16:00',
                      '2010-03-25T22:51:00',
                      '2010-03-28T19:51:00',
                      '2010-04-09T00:51:00',
                      '2010-04-16T18:51:00',
                      '2010-04-24T23:51:00',
                      '2010-04-26T08:51:00',
                      '2010-04-27T10:51:00',
                      '2010-05-02T23:51:00',
                      '2010-05-03T13:51:00',
                      '2010-05-08T08:15:00',
                      '2010-05-11T22:51:00',
                      '2010-05-12T06:51:00',
                      '2010-05-14T09:32:00',
                      '2010-05-18T03:51:00',
                      '2010-05-24T12:51:00',
                      '2010-06-09T09:51:00',
                      '2010-06-10T12:51:00',
                      '2010-06-13T13:51:00',
                      '2010-06-16T23:51:00',
                      '2010-06-22T17:51:00',
                      '2010-07-13T11:13:00',
                      '2010-07-14T08:33:00',
                      '2010-07-14T22:49:00',
                      '2010-07-19T08:05:00',
                      '2010-07-23T09:51:00',
                      '2010-07-23T19:41:00',
                      '2010-07-25T13:51:00',
                      '2010-07-29T05:51:00',
                      '2010-07-29T11:51:00',
                      '2010-08-12T08:06:00',
                      '2010-08-15T13:51:00',
                      '2010-08-16T01:51:00',
                      '2010-08-16T16:51:00',
                      '2010-08-22T12:51:00',
                      '2010-08-23T03:59:00',
                      '2010-08-24T21:51:00',
                      '2010-09-12T13:51:00',
                      '2010-09-13T16:51:00',
                      '2010-09-16T15:51:00',
                      '2010-09-22T18:53:00',
                      '2010-09-27T06:21:00',
                      '2010-09-28T10:00:00',
                      '2010-09-30T03:51:00',
                      '2010-10-01T01:53:00',
                      '2010-10-01T13:53:00',
                      '2010-10-04T02:51:00',
                      '2010-10-05T22:51:00',
                      '2010-10-11T17:53:00',
                      '2010-10-14T14:53:00',
                      '2010-10-26T22:53:00',
                      '2010-10-27T13:51:00',
                      '2010-10-27T19:25:00',
                      '2010-10-27T23:51:00',
                      '2010-11-04T01:53:00',
                      '2010-11-07T12:51:00',
                      '2010-11-10T08:51:00',
                      '2010-11-15T18:36:00',
                      '2010-11-16T03:48:00',
                      '2010-11-16T20:06:00',
                      '2010-11-25T12:53:00',
                      '2010-11-26T00:13:00',
                      '2010-11-30T11:53:00',
                      '2010-12-01T01:47:00',
                      '2010-12-01T20:53:00',
                      '2010-12-12T00:53:00',
                      '2010-12-12T16:00:00',
                      '2010-12-26T09:51:00',                  
                      '2010-3-14T05:00:00',
                      '2010-11-07T05:00:00', 
                     ]

pot_boundary_list = [datetime.strptime(str_date, '%Y-%m-%dT%H:%M:%S') for str_date in orig_boundary_list]

pot_boundary_list_dst = [x + timedelta(hours=+1) if (datetime(2010, 3, 14, 2, 0, 0) <= x < datetime(2010, 11, 7, 1, 0, 0)) else x for x in pot_boundary_list]

pot_boundary_list_dst
#df['pickup_datetime_stdtime'] = df['pickup_datetime'].apply(lambda x: (x + timedelta(hours=-1)) if ((x >= datetime(2010, 3, 14, 3, 0, 0)) and (x < datetime(2010, 11, 7, 2, 0, 0))) else x, meta=('pickup_datetime', 'datetime64[ns]'))

ddf_list = []

for idx, b in enumerate(pot_boundary_list_dst):
    df_d = df[ \
          ((df['pickup_datetime'] >= (b + timedelta(days=-.25-7))) & (df['pickup_datetime'] < (b + timedelta(days=.25-7)))) \
          | ((df['pickup_datetime'] >= (b + timedelta(days=-.25))) & (df['pickup_datetime'] < (b + timedelta(days=.25)))) \
          | ((df['pickup_datetime'] >= (b + timedelta(days=-.25+7))) & (df['pickup_datetime'] < (b + timedelta(days=.25+7))))         
         ]
    
    df_d['b_id'] = idx
    df_d['b_ts'] = b

    
    ddf_list.append(df_d)
    
# https://sourcecodequery.com/example-method/dask.concat
df_d = dd.concat(ddf_list)

df_d['pickup_datetime_1min'] = df_d['pickup_datetime'].dt.round('1min')

df_3w = df_d.groupby(['b_id', 'b_ts', 'pickup_datetime_1min'])['vendor_id'].count().compute()#.apply(lambda x: x.value_counts(), meta=pd.Series(dtype='int', name='vendor_id')).compute()

df_3w = pd.DataFrame(df_3w).reset_index()
df_3w.columns = ['b_id', 'b_ts', 'pickup_datetime_1min', 'rides_per_minute']







#df_all = dd.concat(ddf_list)

#     #df_d = df_d.compute()

#     # https://stackoverflow.com/questions/3463930/how-to-round-the-minute-of-a-datetime-object
#     df_d['pickup_datetime_1min'] = df_d['pickup_datetime'].dt.round('1min')

#     df_3w = df_d.groupby('pickup_datetime_1min', as_index=False).agg({'vendor_id':'count'}).iloc[:,:]
#     df_3w['which_week'] = df_3w['pickup_datetime_1min'].apply(parse_weeks)

#     # https://stackoverflow.com/questions/31226142/python-pandas-delete-the-first-row-by-group
#     df_3wfl = df_3w.groupby('which_week').apply(lambda group: group.iloc[1:-1, :])
#     #df_3w.columns = ['pickup_datetime_1min', 'vendor_id', 'which_week2']

#     # https://thispointer.com/pandas-delete-last-column-of-dataframe-in-python/
#     df_3wfl = df_3wfl.iloc[: , :-1].reset_index()
#     del df_3wfl['level_1']

#     df_3wfl['sequence']=df_3wfl.groupby('which_week').cumcount()+1
    

#     df_all = dd.concat([df_all, df_3wfl], axis=0)
    


#df_all.head()

#df_all.head()




Starting: 2021-05-01 22:14:13.240288


distributed.nanny - WARNING - Worker process still alive after 4 seconds, killing


In [81]:
df_3w

NameError: name 'df_3w' is not defined

In [361]:
def parse_weeks(x):
    b = df_3w.loc[x.index, 'b_ts']
    if x < (b + timedelta(days=-.25)):
        return 'Week before'
    elif x > (b + timedelta(days=+.25)):
        return 'Week after'
    else:
        return 'Current week'

In [68]:
def comp_dates(series, dt):
    if series < (dt + timedelta(days=-.25)):
        return 'Week before'
    elif series > (dt + timedelta(days=+.25)):
        return 'Week after'
    else:
        return 'Current week'
    
# https://stackoverflow.com/questions/34099684/how-to-use-groupby-transform-across-multiple-columns
def find_which_week(x):          
    b = x['b_ts'].iloc[0]
    # here a series is being passed to the comp_dates function through 'apply'
    x['which_week'] = x['pickup_datetime_1min'].apply(comp_dates, args=([b]))    
    return x


# https://stackoverflow.com/questions/3463930/how-to-round-the-minute-of-a-datetime-object
# df_d['pickup_datetime_1min'] = df_d['pickup_datetime'].dt.round('1min')


# df_3w = df_d.value_counts(subset=['b_id', 'b_ts', 'pickup_datetime_1min'], sort=False).to_frame('counts').reset_index()

# here a dataframe is being passed to the function in 'apply'
df_3w = df_3w.groupby(['b_id']).apply(find_which_week)

# remove first and last entry per week in attached to a boundary_id
# because rounding causes each extreme timestamp to be undercounted
df_3wfl = df_3w.groupby(['b_id', 'which_week'], as_index=False).apply(lambda group: group.iloc[1:-1, :])



df_3wfl = df_3wfl.sort_values(by=['b_id', 'pickup_datetime_1min']).reset_index(drop=True)

df_3wfl['sequence']=df_3wfl.groupby(['b_id', 'which_week']).cumcount()+1


# https://stackoverflow.com/questions/1345827/how-do-i-find-the-time-difference-between-two-datetime-objects-in-python
duration = datetime.now() - start_time 
duration_in_s = duration.total_seconds()
minutes = divmod(duration_in_s, 60)[0]


# 3 took 6 minues
# 6 took 9 minutes
# 12 took 19 minutes
# 24 took 35 minutes

print(str(len(orig_boundary_list)) + ' dates took:' + str(minutes) + ' minutes.' )




NameError: name 'df_3w' is not defined

In [76]:
df_3wfl.to_csv('data/3wboundaries-weather-dst-adj.csv', index=False )

In [79]:
# only to read from csv

df_3wfl = pd.read_csv('data/3wboundaries-weather-dst-adj.csv')
del df_3wfl['Unnamed: 0']
df_3wfl
df_3wfl['b_ts'] = pd.to_datetime(df_3wfl['b_ts'], format='%Y-%m-%d %H:%M:%S')
df_3wfl['pickup_datetime_1min'] = pd.to_datetime(df_3wfl['pickup_datetime_1min'], format='%Y-%m-%d %H:%M:%S')
df_3wfl.dtypes

b_id                             int64
b_ts                    datetime64[ns]
pickup_datetime_1min    datetime64[ns]
rides_per_minute                 int64
which_week                      object
sequence                         int64
dtype: object

In [124]:

df_3wfl.head()

,b_id,b_ts,pickup_datetime_1min,rides_per_minute,which_week,sequence,b_ts_str,pickup_datetime_1min_str
0,0,2010-01-17 11:51:00,2010-01-10 05:52:00,55,Week before,1,2010-01-17 11:51:00,2010-01-10 05:52:00
1,0,2010-01-17 11:51:00,2010-01-10 05:53:00,63,Week before,2,2010-01-17 11:51:00,2010-01-10 05:53:00
2,0,2010-01-17 11:51:00,2010-01-10 05:54:00,63,Week before,3,2010-01-17 11:51:00,2010-01-10 05:54:00
3,0,2010-01-17 11:51:00,2010-01-10 05:55:00,71,Week before,4,2010-01-17 11:51:00,2010-01-10 05:55:00
4,0,2010-01-17 11:51:00,2010-01-10 05:56:00,66,Week before,5,2010-01-17 11:51:00,2010-01-10 05:56:00


In [122]:
def make_same_dates(x):          
    print()
    # here a series is being passed to the comp_dates function through 'apply'
    x['pickup_datetime_same_time'] = x.apply(
        lambda row: 0 if (row['which_week'] == 'Week before') else \
        1 if (row['which_week'] == 'Week after') else 2
    )   
    return x

In [123]:
df_3wfl.groupby('b_id').apply(make_same_dates)

KeyError: 'which_week'

In [119]:
df_3wfl['b_ts_str'] = df_3wfl['b_ts'].astype(str)
df_3wfl['pickup_datetime_1min_str'] = df_3wfl['pickup_datetime_1min'].astype(str)
df_3wfl.dtypes

b_id                                 int64
b_ts                        datetime64[ns]
pickup_datetime_1min        datetime64[ns]
rides_per_minute                     int64
which_week                          object
sequence                             int64
b_ts_str                            object
pickup_datetime_1min_str            object
dtype: object

In [125]:


for b in df_3wfl['b_ts'].unique()[0:5]:
    
    # https://stackoverflow.com/questions/13703720/converting-between-datetime-timestamp-and-datetime64
    b = pd.Timestamp(b)
    
    df_single = df_3wfl[df_3wfl['b_ts'] == b]    

    #display(df_single)

    #line = alt.Chart(pd.DataFrame({'x': [359.5]})).mark_rule().encode(x=alt.X('x:Q', title='') )
    line = alt.Chart(pd.DataFrame({'x': [str(b)]})).mark_rule().encode(x=alt.X('utchoursminutes(x):T', title='') )
    #line = alt.Chart(pd.DataFrame({'x': [b]})).mark_rule().encode(x=alt.X('x:T', title='') )

    # https://altair-viz.github.io/user_guide/times_and_dates.html
    rides = alt.Chart(df_single, title="Rides around " + str(b.strftime('%A')) + ' - ' + str(b)).mark_line().encode(
        #x=alt.X('hoursminutes(pickup_datetime_1min):Q', title='Date/time', 
        x=alt.X('utchoursminutes(pickup_datetime_1min_str):T', title='Date/time', 
        #x=alt.X('pickup_datetime_1min_str:T', title='Date/time', 
#                 axis=alt.Axis(
#                             tickCount=20,
#                             grid=False,
#                             labelExpr="datum.value % 1 ? null : datum.label"
#                         )
               ), 
        #x=alt.X('sequence:Q', axis=None, title='Date/time' ),     
        y=alt.Y('rides_per_minute:Q', title='Rides per minute'),
        color=alt.Color('which_week:N', title='Which week', scale=alt.Scale(
                                                domain=['Week before', 'Current week', 'Week after'],
                                                range=['#c0c0c0 ', '#fff400', '#adc0d8']))
    ).properties(
    width=700,
        height=250
    ).interactive()
    
#     text = alt.Chart(df_single).mark_text(
#                         align='center', 
#                         fontSize=10,
#                         color='orange',
#                         baseline='bottom', 
#                         dy=135,
#                         angle=0,
#                         clip=False
        
                        
#                     ).encode(
#                         #y=alt.value(250),
#                         x=alt.X('sequence:Q', axis=None),
#                         text='hoursminutes(pickup_datetime_1min):O',
#                     )

    (rides+line).display()



alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

In [82]:
b = datetime(2010, 1, 17, 11, 51, 0)
b = datetime(2010, 1, 25, 6, 0, 0)

# nothing
b = datetime(2010, 3, 28, 19, 51, 0)
#nothing
b = datetime(2010, 3, 30, 2, 51, 0)
#nothing
b = datetime(2010, 4, 9, 0, 51, 0)
# nothing
b = datetime(2010, 4, 16, 18, 51, 0)
#maybe a bit later
b = datetime(2010, 4, 27, 10, 51, 0)

# spike soon after
b = datetime(2010, 5, 14, 9, 32, 0)
# spike soon after
b = datetime(2010, 5, 27, 17, 51, 0)
# elevated here after
b = datetime(2010, 6, 9, 9, 51, 0)
# small spike after
b = datetime(2010, 6, 10, 12, 51, 0)

# theres a little spike here
b = datetime(2010, 6, 17, 0, 10, 0)

# big spike
b = datetime(2010, 6, 22, 17, 51, 0)

# big spike
b = datetime(2010, 7, 13, 11, 13, 0)

#maybe? minimal
b = datetime(2010, 7, 14, 4, 51, 0)

#maybe
b = datetime(2010, 7, 14, 22, 10, 0)

# small jump
b = datetime(2010, 7, 19, 8, 5, 0)

#maybe
b = datetime(2010, 7, 23, 9, 51, 0)


b = datetime(2010, 6, 22, 17, 51, 0)

b = datetime(2010, 6, 9, 9, 51, 0)
def parse_weeks(x):
    if x < (b + timedelta(days=-.25)):
       return 'Week before'
    elif x > (b + timedelta(days=+.25)):
       return 'Week after'
    else:
       return 'Current week'


df_d = df[ \
      ((df['pickup_datetime'] >= (b + timedelta(days=-.25-7))) & (df['pickup_datetime'] < (b + timedelta(days=.25-7)))) \
      | ((df['pickup_datetime'] >= (b + timedelta(days=-.25))) & (df['pickup_datetime'] < (b + timedelta(days=.25)))) \
      | ((df['pickup_datetime'] >= (b + timedelta(days=-.25+7))) & (df['pickup_datetime'] < (b + timedelta(days=.25+7))))         
     ]

df_d = df_d.compute()

# https://stackoverflow.com/questions/3463930/how-to-round-the-minute-of-a-datetime-object
df_d['pickup_datetime_1min'] = df_d['pickup_datetime'].dt.round('1min')

df_3w = df_d.groupby('pickup_datetime_1min', as_index=False).agg({'vendor_id':'count'}).iloc[:,:]
df_3w['which_week'] = df_3w['pickup_datetime_1min'].apply(parse_weeks)

# https://stackoverflow.com/questions/31226142/python-pandas-delete-the-first-row-by-group
df_3wfl = df_3w.groupby('which_week').apply(lambda group: group.iloc[1:-1, :])
#df_3w.columns = ['pickup_datetime_1min', 'vendor_id', 'which_week2']

# https://thispointer.com/pandas-delete-last-column-of-dataframe-in-python/
df_3wfl = df_3wfl.iloc[: , :-1].reset_index()
del df_3wfl['level_1']

df_3wfl['sequence']=df_3wfl.groupby('which_week').cumcount()+1

#display(df_3wfl)


df_3wfl.to_parquet('./data/3wksWindow' + b.strftime('%Y-%m-%d') + '.parquet')
df_3wfl = pd.read_parquet('./data/3wksWindow' + b.strftime('%Y-%m-%d') + '.parquet', engine='fastparquet')




alt.data_transformers.disable_max_rows()

line = alt.Chart(pd.DataFrame({'x': [b]})).mark_rule().encode(x=alt.X('hoursminutes(x):O', title='') )

# https://altair-viz.github.io/user_guide/times_and_dates.html
rides = alt.Chart(df_3wfl, title="Rides around " + str(b.strftime('%A')) + ' - ' + str(b)).mark_line().encode(
    x=alt.X('hoursminutes(pickup_datetime_1min):O', title='Date/time'),        
    y=alt.Y('vendor_id:Q', title='Rides per minute'),
    color=alt.Color('which_week:N', title='Which week')
).properties(
width=700,
    height=400
).interactive()

rides+line
#     return line+rides

# plot3wksWindow(df, b)

alt.LayerChart(...)

In [83]:
df.head()


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2010-01-26 07:41:00,2010-01-26 07:45:00,1,0.75,-73.956778,40.767750,1,NaN,-73.965957,40.765232,CAS,4.5,0.0,0.5,0.00,0.0,5.00
1,DDS,2010-01-30 23:31:00,2010-01-30 23:46:12,1,5.90,-73.996118,40.763932,1,NaN,-73.981512,40.741193,CAS,15.3,0.5,0.5,0.00,0.0,16.30
2,DDS,2010-01-18 20:22:20,2010-01-18 20:38:12,1,4.00,-73.979673,40.783790,1,NaN,-73.917852,40.878560,CAS,11.7,0.5,0.5,0.00,0.0,12.70
3,VTS,2010-01-09 01:18:00,2010-01-09 01:35:00,2,4.70,-73.977922,40.763997,1,NaN,-73.923908,40.759725,CAS,13.3,0.5,0.5,0.00,0.0,14.30
4,CMT,2010-01-18 19:10:14,2010-01-18 19:17:07,1,0.60,-73.990924,40.734682,1,0,-73.995511,40.739088,Cre,5.3,0.0,0.5,0.87,0.0,6.67


In [55]:
ts_list = [d1s, d1s + timedelta(minutes=20), d1s + timedelta(minutes=10), d1s + timedelta(minutes=30) ]

[str(x.hour) + ':' + str(x.minute) for x in ts_list]

['23:50', '0:10', '0:0', '0:20']

In [105]:
b = datetime(2010, 6, 9, 0, 5, 0)
b_ts = [b] * 12    
    
d1s = datetime(2010, 6, 1, 23, 50, 0)
d2s = datetime(2010, 6, 8, 23, 50, 0)
d3s = datetime(2010, 6, 15, 23, 50, 0)

pickup_datetime_1min = [d1s, d1s + timedelta(minutes=10), d1s + timedelta(minutes=20), d1s + timedelta(minutes=30) ] + \
                        [d2s, d2s + timedelta(minutes=10), d2s + timedelta(minutes=20), d2s + timedelta(minutes=30) ] + \
                        [d3s, d3s + timedelta(minutes=10), d3s + timedelta(minutes=20), d3s + timedelta(minutes=30) ]

pickup_datetime_same = [d2s, d2s + timedelta(minutes=10), d2s + timedelta(minutes=20), d2s + timedelta(minutes=30)] + \
                        [d2s, d2s + timedelta(minutes=10), d2s + timedelta(minutes=20), d2s + timedelta(minutes=30) ] + \
                        [d2s, d2s + timedelta(minutes=10), d2s + timedelta(minutes=20), d2s + timedelta(minutes=30) ]

rides_per_minute = [10,12,14,16] + [12,14,16,18] + [14,16,18,20]

which_week = ["Week before"] * 4 + ['Current week'] * 4 + ['Week after'] * 4
 
sequence = [1,2,3,4] * 3
    
list_of_tuples = list(zip(b_ts, pickup_datetime_1min, pickup_datetime_same, rides_per_minute, which_week, sequence))   
 
dff = pd.DataFrame(list_of_tuples,
                  columns = ['b_ts', 'pickup_datetime_1min', 'pickup_datetime_same', 'rides_per_minute', 'which_week', 'sequence']) 

dff.dtypes

b_ts                    datetime64[ns]
pickup_datetime_1min    datetime64[ns]
pickup_datetime_same    datetime64[ns]
rides_per_minute                 int64
which_week                      object
sequence                         int64
dtype: object

In [91]:
line = alt.Chart(pd.DataFrame({'x': [b]})).mark_rule().encode(x=alt.X('hoursminutes(x):O', title='') )
#line = alt.Chart(pd.DataFrame({'x': [b]})).mark_rule().encode(x=alt.X('x:O', title='') )

# https://altair-viz.github.io/user_guide/times_and_dates.html
rides = alt.Chart(dff, title="Rides around " + str(b.strftime('%A')) + ' - ' + str(b)).mark_line().encode(
    x=alt.X('utchoursminutes(pickup_datetime_1min):O', title='Date/time'), 
    #x=alt.X('hoursminutes(pickup_datetime_same):O', title='Date/time' ), 
    #x=alt.X('sequence:Q', axis=None, title='Date/time' ),     
    y=alt.Y('rides_per_minute:Q', title='Rides per minute'),
    color=alt.Color('which_week:N', title='Which week', scale=alt.Scale(
                                            domain=['Week before', 'Current week', 'Week after'],
                                            range=['#c0c0c0 ', '#fff400', '#adc0d8']))
).properties(
width=700,
    height=250
).interactive()

(rides+line).display()

alt.LayerChart(...)

In [104]:
line = alt.Chart(pd.DataFrame({'x': [b]})).mark_rule().encode(x=alt.X('utchoursminutes(x):O', title='') )
#line = alt.Chart(pd.DataFrame({'x': [b]})).mark_rule().encode(x=alt.X('x:T', title='') )

# https://altair-viz.github.io/user_guide/times_and_dates.html
rides = alt.Chart(dff, title="Rides around " + str(b.strftime('%A')) + ' - ' + str(b)).mark_line().encode(
    #x=alt.X('hoursminutes(pickup_datetime_1min):Q', title='Date/time', 
    x=alt.X('utchoursminutes(pickup_datetime_1min):O', title='Date/time', 
            
           ), 
    #x=alt.X('sequence:Q', axis=None, title='Date/time' ),     
    y=alt.Y('rides_per_minute:Q', title='Rides per minute'),
    color=alt.Color('which_week:N', title='Which week', scale=alt.Scale(
                                            domain=['Week before', 'Current week', 'Week after'],
                                            range=['#c0c0c0 ', '#fff400', '#adc0d8']))
).properties(
width=700,
    height=250
).interactive()

(rides+line).display()

alt.LayerChart(...)